In [ ]:
%pip install Faker

### Initialize Customer Profiles
<div class="alert alert-info">Update the following information in the <i>initialize.py</i> file:
<ul>
    <li>Replace the <b>domainName</b> with your customer profile domain. It can be found within the AWS Console -> Amazon Connect -> Customer Profiles as seen in the below screenshot
    <li>Replace the AWS Region in the endpoint url to match the region in which you created the Customer Profile domain</li>
</ul>
</div>

![domain](../assets/cp_domain.png)

### Import Python libraries

In [ ]:
import dg_functions
import json
import uuid
import boto3
import initialize as cp

### Generate Loyalty, Preferences, and Air Bookings Objects

In [ ]:
dataObj = dg_functions.generateData()
print(f"LoyaltyID: {dataObj['Loyalty']['LoyaltyId']}")
print(f"No of Bookings: {len(dataObj['Booking'])}")

### Send the Loyalty Object to Customer Profiles

In [ ]:
cp.client.put_profile_object(
    DomainName = cp.domainName,
    ObjectTypeName = 'Loyalty',
    Object = json.dumps(dataObj['Loyalty'])
)

### Send the preferences data to Customer Profiles

In [ ]:
cp.client.put_profile_object(
    DomainName = cp.domainName,
    ObjectTypeName = 'Preferences',
    Object = json.dumps(dataObj['Preferences'])
)

### Send the booking records to Kinesis

In [ ]:
kinesis_client = boto3.client('kinesis')
stream_name = 'airlines-input-cp-stream'
partition_key = str(uuid.uuid4())

In [ ]:
# Send each booking record to Kinesis data stream
bookings = dataObj['Booking']
for booking in bookings:
    data_as_bytes = json.dumps(booking).encode('utf-8')
    response = kinesis_client.put_record(
        StreamName=stream_name,
        Data=data_as_bytes,
        PartitionKey=partition_key
    )